# Imports

In [3]:
import os 
import tarfile
import urllib.request 

In [15]:
import email
import email.policy

In [26]:
from collections import Counter

In [33]:
import pandas as pd

In [65]:
import numpy as np
from sklearn.model_selection import train_test_split

In [68]:
import re 
from html import unescape 

In [78]:
import nltk

In [82]:
import urlextract

In [85]:
from sklearn.base import BaseEstimator, TransformerMixin

In [94]:
from scipy.sparse import csr_matrix

# Spam Classifier Notebook

Chapter 3 Question 4 (page 109)

## Fetching Data

- fetch data and store in raw_data/

In [10]:
spam_assassin_path = 'http://spamassassin.apache.org/old/publiccorpus/'
ham_url = spam_assassin_path + '20030228_easy_ham.tar.bz2'
spam_url = spam_assassin_path + '20030228_spam.tar.bz2'

raw_data_path = '../raw_data/'
spam_path = os.path.join(raw_data_path, 'spam')

def fetch_data(ham_url=ham_url, spam_url=spam_url, raw_data_path=raw_data_path):
    # Check that path to spam folder in raw_data exists. If not, then create directory 
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    
    for filename, url in (('ham.tar.bz2', ham_url), ('spam.tar.bz2', spam_url)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()

In [6]:
fetch_data()

## Load emails into notebook 

In [11]:
spam_path

'../raw_data/spam'

In [12]:
ham_dir = os.path.join(spam_path, 'easy_ham')
spam_dir = os.path.join(spam_path, 'spam')

ham_filenames = [name for name in sorted(os.listdir(ham_dir)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(spam_dir)) if len(name) > 20]

In [13]:
len(ham_filenames)

2500

In [14]:
len(spam_filenames)

500

## Data Prep

- Use Python `email` module to parse emails 

In [16]:
def load_email(is_spam, filename, spam_path=spam_path):
    # Directory dependent on wether passed email is spam or not 
    directory = 'spam' if is_spam else 'easy_ham'
    # Open required file in reading mode (binary)
    with open(os.path.join(spam_path, directory ,filename), 'rb') as f:
        # Returned parsed email.message
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [18]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [22]:
# get_content() - returns the 'payload' from the email
# stip() - removes white space
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [23]:
print(spam_emails[1].get_content().strip())

1) Fight The Risk of Cancer!
http://www.adclick.ws/p.cfm?o=315&s=pk007

2) Slim Down - Guaranteed to lose 10-12 lbs in 30 days
http://www.adclick.ws/p.cfm?o=249&s=pk007

3) Get the Child Support You Deserve - Free Legal Advice
http://www.adclick.ws/p.cfm?o=245&s=pk002

4) Join the Web's Fastest Growing Singles Community
http://www.adclick.ws/p.cfm?o=259&s=pk007

5) Start Your Private Photo Album Online!
http://www.adclick.ws/p.cfm?o=283&s=pk007

Have a Wonderful Day,
Offer Manager
PrizeMama













If you wish to leave this list please use the link below.
http://www.qves.com/trim/?ilug@linux.ie%7C17%7C114258


-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie


### Investigating different structures of emails:

In [25]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return f"mulitipart({', '.join([get_email_structure(sub_email) for sub_email in payload])})"
    else:
        return email.get_content_type()

In [27]:
def stuctures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [36]:
ham_structure = stuctures_counter(ham_emails).most_common()

In [32]:
spam_structure = stuctures_counter(spam_emails).most_common()

In [46]:
ham_structure_df = pd.DataFrame(ham_structure, columns=['structure', 'ham_count'])
spam_structure_df = pd.DataFrame(spam_structure, columns=['structure', 'spam_count'])
structure_df = ham_structure_df.merge(spam_structure_df, how='outer').fillna(0)
structure_df['ham_percent'] = structure_df['ham_count'] / len(ham_emails)
structure_df['spam_percent'] = structure_df['spam_count'] / len(spam_emails)


In [60]:
structure_df.sort_values(by=['spam_percent'], ascending=False).head(5)

,structure,ham_count,spam_count,ham_percent,spam_percent
0,text/plain,2408.0,218.0,0.9632,0.436
15,text/html,0.0,183.0,0.0000,0.366
2,"mulitipart(text/plain, text/html)",8.0,45.0,0.0032,0.090
16,mulitipart(text/html),0.0,20.0,0.0000,0.040
4,mulitipart(text/plain),3.0,19.0,0.0012,0.038


In [61]:
structure_df.sort_values(by=['ham_percent'], ascending=False).head(5)

,structure,ham_count,spam_count,ham_percent,spam_percent
0,text/plain,2408.0,218.0,0.9632,0.436
1,"mulitipart(text/plain, application/pgp-signature)",66.0,0.0,0.0264,0.000
2,"mulitipart(text/plain, text/html)",8.0,45.0,0.0032,0.090
3,"mulitipart(text/plain, text/plain)",4.0,0.0,0.0016,0.000
4,mulitipart(text/plain),3.0,19.0,0.0012,0.038


- Ham emails contain mostly text/plain with 96.32%
- Ham emails are more likely to contain a pgp-signature 2.64% compared to spam's 0%
- Spam has a high rate of text/html of 36.6% compared to ham's 0%

Inspecting email headers:


In [63]:
for header, value in spam_emails[0].items():
    print(header, ':__ ', value)

Return-Path :__  <12a1mailbot1@web.de>
Delivered-To :__  zzzz@localhost.spamassassin.taint.org
Received :__  from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received :__  from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received :__  from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From :__  12a1mailbot1@web.de
Received :__  from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To :__  dcek1a1@netsgo.com
Subject :__  Life Insurance - Why Pay More?
Date :__  Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version :__  1.0
Message-ID :__  <0103c1042001882DD_IT7@dd_it7>
Content-Type :__  text/html; charset="iso

- can look at sender email through 'Return-Path' (12almailbot1@web.de looks suspicious)
- Focus on `Subject`

In [64]:
spam_emails[0]['Subject']

'Life Insurance - Why Pay More?'

## Splitting into Train and Test Set

In [67]:
X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0]*len(ham_emails) + [1]*len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preprocessing Pipeline

In [73]:
def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)



html_spam_emails = [email for email in X_train[y_train==1] if get_email_structure(email) == 'text/html']

sample_html_spam = html_spam_emails[7]


In [74]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [75]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ('text/plain', 'text/html'):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == 'text/plain':
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [76]:
print(email_to_text(sample_html_spam)[:100], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Wat ...


In [80]:
stemmer = nltk.PorterStemmer()
for word in ('Computations', 'Computation', 'Computing', 'Computed','Compute','Compulsive'):
    print(word, '=> ', stemmer.stem(word))
    
stemmer = None

Computations =>  comput
Computation =>  comput
Computing =>  comput
Computed =>  comput
Compute =>  comput
Compulsive =>  compuls


In [84]:
url_extractor = urlextract.URLExtract()
print(url_extractor.find_urls('test github.com test test https://docs.python.org/3/library/re.html#re.M'))

['github.com', 'https://docs.python.org/3/library/re.html#re.M']


## Building a Transformer 

In [92]:
class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ''
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, ' URL ')
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [93]:
stemmer = nltk.PorterStemmer()
url_extractor = urlextract.URLExtract()
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

In [105]:
class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
        
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X: 
            # eg Counter{'chuck': 1, 'murcko': 1}, Counter({'the': 11, 'of': 9}
            for word, count in word_count.items(): 
                # eg 'chuck', 1   'murcko': 1
                total_count[word] += min(count, 10) # max 10, takes minimum of count or 10
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word: index+1 for index, (word, count) in enumerate(most_common)}
        return self
    
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size+1))
                    

In [106]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [107]:
X_few_vectors.toarray()

array([[ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [99, 11,  9,  8,  3,  1,  3,  1,  3,  2,  3],
       [67,  0,  1,  2,  3,  4,  1,  2,  0,  1,  0]])

In [108]:
vocab_transformer.vocabulary_

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'url': 5,
 'all': 6,
 'in': 7,
 'christian': 8,
 'on': 9,
 'by': 10}

## Training Full dataset

In [109]:
from sklearn.pipeline import Pipeline

In [110]:
preprocess_pipeline = Pipeline([
    ('email_to_wordcount', EmailToWordCounterTransformer()),
    ('wordcount_to_vector', WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] .................................... , score=0.981, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.984, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.990, total=   0.2s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished


0.985

In [113]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))

Precision: 96.88%
Recall: 97.89%
[CV]  ................................................................
[CV] .................................... , score=0.981, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.984, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.990, total=   0.2s
